In [9]:
# Bidirectional Brute Force
# https://www.kaggle.com/code/davidlist/bidirectional-brute-force

# The difficulty with brute force methods in most all of these problems is the extent with
# which the number of permutations explodes exponentially with the distance from any given state.
# However, by proceeding outwards from both the initial state and the solution state with
# the hope of meeting in the middle, it's possible to limit the permutations enough
# to find optimal solutions for a few of the puzzles.

In [10]:
import pandas as pd
import numpy as np
import ast
import os
import time

In [11]:
PUZZLE_FILE = './raw_data/puzzles.csv'
PUZZLE_INFO_FILE = './raw_data//puzzle_info.csv'
# SAMPLE_FILE = './raw_data/sample_submission.csv'
SAMPLE_FILE = './submission.csv'

RUN_TIME = 60 * 60 * 11
START_TIME = time.time()
TIMEOUT = 60 * 5

INCLUDES = list(range(398))
# EXCLUDES = [
#     286,
#     291,
#     285,
#     287,
#     309,
#     310,
# ]
MIN_SIZE = None
MAX_SIZE = None

DEBUG = False

In [12]:
data_df = pd.read_csv(PUZZLE_FILE)
info_df = pd.read_csv(PUZZLE_INFO_FILE)
sample_df = pd.read_csv(SAMPLE_FILE)

In [13]:
def init_reverse_moves(moves):
    new_moves = {}

    for m in moves.keys():
        new_moves[m] = moves[m]
        xform = moves[m]
        m_inv = '-' + m
        xform_inv = len(xform) * [0]
        for i in range(len(xform)):
            xform_inv[xform[i]] = i
        new_moves[m_inv] = xform_inv

    return new_moves

def apply_move(move, state):

    m = move
    s = state.split(';')

    move_list = moves[m]
    new_state = []
    for i in move_list:
        new_state.append(s[i])
    s = new_state

    return ';'.join(s)

def reverse_move(move, state):
    m = move[1:] if move[0] == '-' else '-' + move
    return apply_move(m, state)

def expand(paths, reverse=False):
    states = list(paths.keys())
    for s in states:
        for m in moves:
            if reverse:
                next_s = reverse_move(m, s)
                if not next_s in paths:
                    paths[next_s] = paths[s] + [m]
            else:
                next_s = apply_move(m, s)
                if not next_s in paths:
                    paths[next_s] = paths[s] + [m]

In [14]:
def solve(pid):
    global moves, source_paths, dest_paths

    ddf = data_df[data_df['id'] == pid]
    puzzle_type = ddf['puzzle_type'].iloc[0]
    solution_state = ddf['solution_state'].iloc[0]
    initial_state = ddf['initial_state'].iloc[0]
    num_wildcards = ddf['num_wildcards'].iloc[0]

    idf = info_df[info_df['puzzle_type'] == puzzle_type]
    allowed_moves = idf['allowed_moves'].iloc[0]
    moves = ast.literal_eval(allowed_moves)

    moves = init_reverse_moves(moves)

    if num_wildcards < 2:
        source_paths = {
            initial_state: []
        }

        dest_paths = {
            solution_state: []
        }

        start_time = time.time()
        solution = None
        count = 0
        while time.time() - START_TIME < RUN_TIME and time.time() - start_time < TIMEOUT:
            count += 1

            if count % 2:
                expand(source_paths)
            else:
                expand(dest_paths, reverse=True)

            overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
            if len(overlap) > 0:
                mnsc = 10000000
                mnsl = None
                for oi in range(len(overlap)):
                    ol = overlap[oi]
                    sl = '.'.join(source_paths[ol] + list(reversed(dest_paths[ol])))
                    sz = len(sl.split('.'))
                    if sz < mnsc:
                        mnsc = sz
                        mnsl = sl
                solution = mnsl
                break
    else:
        ssl = solution_state.split(';')
        mn_score = 10000000
        mn_sol = None
        for i in range(len(ssl)):
            for j in range(len(ssl)):
                if j <= i:
                    continue

                sol = None

                ssln = ssl.copy()
                t = ssln[i]
                ssln[i] = ssln[j]
                ssln[j] = t
                ss = ';'.join(ssln)

                source_paths = {
                    initial_state: []
                }

                dest_paths = {
                    ss: []
                }

                start_time = time.time()
                solution = None
                count = 0
                while time.time() - START_TIME < RUN_TIME and time.time() - start_time < TIMEOUT:
                    count += 1

                    if count % 2:
                        expand(source_paths)
                    else:
                        expand(dest_paths, reverse=True)

                    overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
                    if len(overlap) > 0:
                        mnsc = 10000000
                        mnsl = None
                        for oi in range(len(overlap)):
                            ol = overlap[oi]
                            sl = '.'.join(source_paths[ol] + list(reversed(dest_paths[ol])))
                            sz = len(sl.split('.'))
                            if sz < mnsc:
                                mnsc = sz
                                mnsl = sl
                                if DEBUG:
                                    print('=> [A] ' + str(sz) + ' : ' + sl)
                        sol = mnsl
                        break

                if sol is not None:
                    sz = len(sol.split('.'))
                    if sz < mn_score:
                        mn_score = sz
                        mn_sol = sol

                        if DEBUG:
                            print('=> {' + str(sz) + '} ' + str(sol))

        solution = mn_sol

    return solution

In [15]:
moves = None
source_paths = None
dest_paths = None

In [16]:
grows = []
all_ids = sample_df['id'].unique()
for pid in all_ids:
    if INCLUDES is not None and pid not in INCLUDES:
        print('=> [' + str(pid) + '] Not included!')
        continue
    # if EXCLUDES is not None and pid in EXCLUDES:
    #     print('=> [' + str(pid) + '] Excluded!')
    #     continue

    df = data_df[data_df['id'] == pid]
    type = df['puzzle_type'].iloc[0]
    # if not (type.startswith('cube_2/2/2') or type.startswith('wreath')):
    #     continue
    if not type.startswith('globe_1/8'):
        continue
    solution_state = df['solution_state'].iloc[0]
    size = len(solution_state.split(';'))

    if MIN_SIZE is not None and size <= MIN_SIZE:
        print('=> [' + str(pid) + '] Skipped by lower size!')
        continue

    if MAX_SIZE is not None and size >= MAX_SIZE:
        print('=> [' + str(pid) + '] Skipped by upper size!')
        continue

    solution = solve(pid)
    if solution is None:
        print('=> [' + str(pid) + '] Failed!')
    else:
        print('=> [' + str(pid) + '] Success!')

        grows.append({'id': pid, 'moves': solution})

=> [0] Success!
=> [1] Success!
=> [2] Success!
=> [3] Success!
=> [4] Success!
=> [5] Success!
=> [6] Success!
=> [7] Success!
=> [8] Success!
=> [9] Success!
=> [10] Success!
=> [11] Success!
=> [12] Success!
=> [13] Success!
=> [14] Success!
=> [15] Success!
=> [16] Success!
=> [17] Success!
=> [18] Success!
=> [19] Success!
=> [20] Success!
=> [21] Success!
=> [22] Success!
=> [23] Success!
=> [24] Success!
=> [25] Success!
=> [26] Success!
=> [27] Success!
=> [28] Success!
=> [29] Success!
=> [284] Success!
=> [285] Success!
=> [286] Success!
=> [287] Success!
=> [288] Success!
=> [289] Success!
=> [290] Success!
=> [291] Success!
=> [292] Success!
=> [293] Success!
=> [294] Success!
=> [295] Success!
=> [296] Success!
=> [297] Success!
=> [298] Success!
=> [299] Success!
=> [300] Success!
=> [301] Success!
=> [302] Success!
=> [303] Success!
=> [304] Success!
=> [305] Success!
=> [306] Success!
=> [307] Success!
=> [308] Success!
=> [309] Success!
=> [310] Success!
=> [311] Succe

In [17]:
gdf = None
if len(grows) > 0:
    gdf = pd.DataFrame(grows)
    gdf.to_csv('good.csv', index=False)
gdf

,id,moves
0,0,r1.-f1
1,1,f0.-r0.-d0.r0.r0.d1.r1.f0.d0
2,2,d0.f1.-r1.-d1.f1.-r0.-f1.-d0.-r0.f0.d0.d0
3,3,-f0.d0.-r0.f0.d1.f1.-d1.-f0.-r0.-f0
4,4,r0.-d0.r0.d0.-f0.f1.r1.-d0.-f1.-d0.-f1.r0
...,...,...
71,325,-r.l.-r.-l.-l.r.r.-l.-l.r.r.-l.r.-l.r.r.-l.r.r...
72,326,l.-r.-l.-l.-l.r.r.r.l.l.-r.l.l.-r.l.-r.-r.l
73,327,l.r.-l.r.l.-r.-l.-r.-l.-l.-r.l.-r.-l.-r.-r.-r....
74,328,-r.l.l.l.l.-r.-r.l.-r.l.l.r.l.l.r


In [18]:
sub_df = None
if gdf is not None:
    bdf = gdf
    adf = sample_df
    rows = []
    for ri in range(len(adf)):
        pid = adf['id'].iloc[ri]
        moves = adf['moves'].iloc[ri]
        size = len(moves.split('.'))
        df = bdf[bdf['id'] == pid]
        if len(df) > 0:
            moves_b = df['moves'].iloc[0]
            size_b = len(moves_b.split('.'))
            if size_b < size:
                moves = moves_b
                print('=> (' + str(pid) + ') ' + str(size) + ' -> ' + str(size_b))
        rw = {'id': pid, 'moves': moves}
        rows.append(rw)
    sub_df = pd.DataFrame(rows)
    sub_df.to_csv('submission.csv', index=False)
sub_df

=> (1) 63 -> 9
=> (2) 62 -> 12
=> (3) 92 -> 10
=> (4) 70 -> 12
=> (5) 54 -> 10
=> (6) 68 -> 10
=> (7) 83 -> 11
=> (8) 98 -> 10
=> (9) 76 -> 10
=> (10) 66 -> 10
=> (11) 63 -> 9
=> (12) 72 -> 12
=> (13) 131 -> 11
=> (14) 96 -> 12
=> (15) 68 -> 10
=> (16) 63 -> 11
=> (17) 62 -> 10
=> (18) 89 -> 11
=> (19) 82 -> 12
=> (20) 112 -> 12
=> (21) 96 -> 12
=> (22) 63 -> 11
=> (23) 53 -> 9
=> (24) 99 -> 11
=> (25) 61 -> 13
=> (26) 93 -> 11
=> (27) 73 -> 11
=> (28) 83 -> 11
=> (29) 82 -> 10
=> (284) 339 -> 7
=> (285) 335 -> 3
=> (286) 484 -> 5
=> (287) 482 -> 2
=> (288) 489 -> 6
=> (289) 351 -> 8
=> (290) 356 -> 10
=> (291) 380 -> 5
=> (292) 378 -> 8
=> (293) 322 -> 8
=> (294) 341 -> 10
=> (295) 486 -> 9
=> (296) 319 -> 10
=> (297) 452 -> 9
=> (298) 393 -> 7
=> (299) 398 -> 8
=> (300) 327 -> 10
=> (301) 397 -> 5
=> (302) 447 -> 9
=> (303) 412 -> 11
=> (304) 422 -> 10
=> (305) 441 -> 9
=> (306) 428 -> 10
=> (307) 409 -> 10
=> (308) 470 -> 8
=> (309) 408 -> 5
=> (310) 421 -> 3
=> (311) 706 -> 10
=> (

,id,moves
0,0,r1.-f1
1,1,f0.-r0.-d0.r0.r0.d1.r1.f0.d0
2,2,d0.f1.-r1.-d1.f1.-r0.-f1.-d0.-r0.f0.d0.d0
3,3,-f0.d0.-r0.f0.d1.f1.-d1.-f0.-r0.-f0
4,4,r0.-d0.r0.d0.-f0.f1.r1.-d0.-f1.-d0.-f1.r0
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...
